In [57]:
import spotipy
import base64
import requests
from spotipy.oauth2 import SpotifyClientCredentials
from creds import CLIENT_ID, CLIENT_SECRET
import pandas as pd
import time

In [44]:

auth_manager = SpotifyClientCredentials(CLIENT_ID, CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

# credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
# credentials_base64 = base64.b64encode(credentials.encode()) #utf-8

# #Make a request for the access token
# token_url = 'https://accounts.spotify.com/api/token'
# headers = {
#     'Authorization': f'Basic {credentials_base64.decode()}'
# }
# data = {
#     'grant_type': 'client_credentials'
# }
# response = requests.post(token_url, data=data, headers=headers)

# if response.status_code == 200:
#     access_token = response.json()['access_token']
#     print("Access token obtained successfully.")
# else:
#     print("Access token not obtained.")
#     exit()

# def get_auth_token(token):
#     return {"Authorization": "Bearer " + access_token}
# get_auth_token(access_token)

In [ ]:
#Get spotify playlist ids
playlist_ids = []
playlists = sp.user_playlists('spotify')

while playlists: # example provided in spotipy documentation
    for i, playlist in enumerate(playlists['items']):
        print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
        playlist_ids.append(playlist['uri'][-22:])
        #print('IDs appended')
    if playlists['next']:
        playlists = sp.next(playlists)
        
    else:
        playlists = None


In [62]:
# Get track info from playlists into a dataframe

def get_tracks(playlist_id):

    tracks = sp.playlist_tracks(id, limit = 50, fields='items(track(id, name, artists, album(id, name)))')
    music_data = []

    for track_info in tracks['items']:
        track = track_info['track']
    # still want popularity, some audio features and album release dates

        if track:
            track = track_info['track']
            track_name = track['name']
            artists = ', '.join([artist['name'] for artist in track['artists']])
            album_name = track['album']['name']
            album_id = track['album']['id']
            track_id = track['id']
       
        #audio_stuffs = sp.audio_features(track_id)[0] if track_id else None

        track_data = {
            'Track': track,
            'Track Name': track_name,
            'Artist': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id
            # 'Acousticness': audio_stuffs['acousticness'] if audio_stuffs else None,
            # 'Danceability': audio_stuffs['danceability'] if audio_stuffs else None,
            # 'Energy': audio_stuffs['energy'] if audio_stuffs else None,
            # 'Instrumentalness': audio_stuffs['instrumentalness'] if audio_stuffs else None,
            # 'Loudness': audio_stuffs['loudness'] if audio_stuffs else None,
            # 'Speechiness': audio_stuffs['speechiness'] if audio_stuffs else None,
            # 'Tempo': audio_stuffs['tempo'] if audio_stuffs else None,
            # 'Mode': audio_stuffs['mode'] if audio_stuffs else None,
            # 'Valence': audio_stuffs['valence'] if audio_stuffs else None
        }

        music_data.append(track_data)
    return music_data
    
#Call function
all_tracks = []       
for id in playlist_ids:
    print(f"Getting tracks from playlist ID: {id}")
    tracks = get_tracks(id)
    all_tracks.extend(tracks)
    #time.sleep(5) # to keep the spotify api server from giving me a 429 error when they finally take me back

spotify_track_df = pd.DataFrame(all_tracks, columns=['Track Name', 'Artist', 'Album Name', 'Album ID', 'Track ID', 
                                                     'Acousticness', 'Danceability','Energy', 'Instrumentalness', 
                                                     'Loudness', 'SPeechiness', 'Tempo', 'Mode', 'Valence'])

spotify_track_df.head(25)


Getting tracks from playlist ID: 37i9dQZF1DXcBWIGoYBM5M
Getting tracks from playlist ID: 37i9dQZF1DX0XUsuxWHRQd
Getting tracks from playlist ID: 37i9dQZF1DX1lVhptIYRda
Getting tracks from playlist ID: 37i9dQZF1DX10zKzsJ2jva
Getting tracks from playlist ID: 37i9dQZF1DX4JAvHpjipBk
Getting tracks from playlist ID: 37i9dQZF1DX4sWSpwq3LiO
Getting tracks from playlist ID: 37i9dQZF1DX4SBhb3fqCJd
Getting tracks from playlist ID: 37i9dQZF1DWXRqgorJj26U
Getting tracks from playlist ID: 37i9dQZF1DX4dyzvuaRJ0n
Getting tracks from playlist ID: 37i9dQZF1DXcF6B6QPhFDv
Getting tracks from playlist ID: 37i9dQZF1DWXJfnUiYjUKT
Getting tracks from playlist ID: 37i9dQZF1DXcRXFNfZr7Tp
Getting tracks from playlist ID: 37i9dQZF1DX4o1oenSJRJd
Getting tracks from playlist ID: 37i9dQZF1DXbTxeAdrVG2l
Getting tracks from playlist ID: 37i9dQZF1DX4UtSsGT1Sbe
Getting tracks from playlist ID: 37i9dQZF1DWTJ7xPn4vNaz
Getting tracks from playlist ID: 37i9dQZF1DXaKIA8E7WcJj
Getting tracks from playlist ID: 37i9dQZF1DWSV3T

,Track Name,Artist,Album Name,Album ID,Track ID,Acousticness,Danceability,Energy,Instrumentalness,Loudness,SPeechiness,Tempo,Mode,Valence
0,I Had Some Help (Feat. Morgan Wallen),"Post Malone, Morgan Wallen",I Had Some Help,1woYXxyyxTQJ0E0AhZE6mj,7221xIgOnuakPdLqT0F3nP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Espresso,Sabrina Carpenter,Espresso,5quMTd5zeI9yW5UDua8wS4,2qSkIjg1o9h3YT9RAgYN75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LUNCH,Billie Eilish,HIT ME HARD AND SOFT,7aJuG4TFXa2hmE4z1yxc3n,629DixmZGHc7ILtEntuiWE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Too Sweet,Hozier,Unheard,1KZC0cX0qq6hodR9YVgh9F,4IadxL6BUymXlh8RCJJu7T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fortnight (feat. Post Malone),"Taylor Swift, Post Malone",THE TORTURED POETS DEPARTMENT,1Mo4aZ8pdj6L1jx8zSwJnt,2OzhQlSqBEmt7hmkYxfT6m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,A Bar Song (Tipsy),Shaboozey,A Bar Song (Tipsy),6egBeCLeGITzGSo5VyRjwZ,2FQrifJ1N335Ljm3TjTVVf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,MILLION DOLLAR BABY,Tommy Richman,MILLION DOLLAR BABY,52TwRwdTUMtkpglbOE5IRz,7fzHQizxTqy8wTXwlrgPQQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Beautiful Things,Benson Boone,Beautiful Things,29aSKB1qPEbN0Qf9OPSQpw,6tNQ70jh4OwmPGpYy6R2o9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,we can't be friends (wait for your love),Ariana Grande,eternal sunshine,71PfP4E9roOXAudea1aPGw,46kspZSY3aKmwQe7O77fCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Not Like Us,Kendrick Lamar,Not Like Us,5JjnoGJyOxfSZUZtk2rRwZ,6AI3ezQ4o3HUoP6Dhudph3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
